In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Setup path in .env file
import os
from dotenv import load_dotenv
import cv2
import numpy as np
import torch

TODO:
- CER Loss testen
- Optimiser wechseln
- Strukturierung Ordnerstruktur
  - Ordner für alte approaches

In [3]:
from emnist import get_emnist_data_loaders
train_loader, val_loader = get_emnist_data_loaders()

In [22]:
from models.crnn.crnn import CRNN
from models.lstm_cnn.lstm_cnn import CNN_LSTM
import torch.nn as nn
from training.trainer import Trainer
import hyperparameter as hp
import torch

crnn = CRNN(hp.GRU_HIDDEN_SIZE, hp.GRU_NUM_LAYERS, hp.NUM_CLASSES, hp.CNN_OUTPUT_HEIGHT)
lstm = CNN_LSTM(hp.GRU_HIDDEN_SIZE, hp.GRU_NUM_LAYERS, hp.NUM_CLASSES, hp.CNN_OUTPUT_HEIGHT)

models = {
    "CRNN": crnn,
    "CNN_LSTM": lstm
}

crnn_criterion = nn.CTCLoss(blank=hp.BLANK_LABEL, reduction='mean', zero_infinity=True)
lstm_criterion = nn.CTCLoss(blank=hp.BLANK_LABEL, reduction='mean', zero_infinity=True)

criterions = {
    "CRNN": crnn_criterion,
    "CNN_LSTM": lstm_criterion
}

crnn_optimizer = torch.optim.Adam(crnn.parameters(), lr=hp.LEARNING_RATE)
lstm_optimizer = torch.optim.Adam(lstm.parameters(), lr=hp.LEARNING_RATE)

optimizers = {
    "CRNN": crnn_optimizer,
    "CNN_LSTM": lstm_optimizer
}

crnn_trainer = Trainer(models["CRNN"], criterions["CRNN"], optimizers["CRNN"], train_loader, val_loader, hp.EPOCHS, {
        "blank_label": hp.BLANK_LABEL,
        "cnn_output_width": hp.CNN_OUTPUT_WIDTH,
    })
lstm_trainer =  Trainer(models["CNN_LSTM"], criterions["CNN_LSTM"], optimizers["CNN_LSTM"], train_loader, val_loader, hp.EPOCHS, {
        "blank_label": hp.BLANK_LABEL,
        "cnn_output_width": hp.CNN_OUTPUT_WIDTH,
    })
trainers = {
    "CRNN": crnn_trainer,
    "CNN_LSTM": lstm_trainer
}


metrics_global = {}
for model_name, trainer in trainers.items():
    print(f"Training {model_name}")
    metrics = trainer.train_validate_test()
    metrics_global[model_name] = metrics
    print(f"Finished training {model_name}")
    break

Training CRNN
100%|██████████| 125/125 [00:39<00:00,  3.19it/s]
EPOCH 1/1 - TRAINING. Correct: 76/8000 = 0.0095 - Average CER Score: 0.922
100%|██████████| 2000/2000 [00:12<00:00, 159.63it/s]
EPOCH 1/1 - TESTING. Correct: 436/2000 = 0.2180 - Average CER Score: 0.387
Finished training CRNN


In [5]:

# ============================================ TESTING =================================================================
number_of_test_imgs = 10
test_loader = torch.utils.data.DataLoader(val_set, batch_size=number_of_test_imgs, shuffle=True)
test_preds = []
(x_test, y_test) = next(iter(test_loader))
y_pred = model(x_test.view(x_test.shape[0], 1, x_test.shape[1], x_test.shape[2]))
y_pred = y_pred.permute(1, 0, 2)
_, max_index = torch.max(y_pred, dim=2)
for i in range(x_test.shape[0]):
    raw_prediction = list(max_index[:, i].detach().cpu().numpy())
    prediction = torch.IntTensor([c for c, _ in groupby(raw_prediction) if c != blank_label])
    test_preds.append(prediction)

for j in range(len(x_test)):
    mpl.rcParams["font.size"] = 8
    plt.imshow(x_test[j], cmap='gray')
    mpl.rcParams["font.size"] = 18
    plt.gcf().text(x=0.1, y=0.1, s="Actual: " + str(y_test[j].numpy()))
    plt.gcf().text(x=0.1, y=0.2, s="Predicted: " + str(test_preds[j].numpy()))
    plt.show()



NameError: name 'val_set' is not defined